# Single Species Hatchery
See [single species size-spectrum dynamics](https://sizespectrum.org/mizer/articles/single_species_size-spectrum_dynamics.html) for model description and basic analysis.

In [ ]:
source("hatchery.R") # imports mizer
library(mizerExperimental)
library(ggplot2)
library(plotly)

theme(
    plot.title = element_text(size = 40),
    axis.text = element_text(size = 40),
    axis.ticks = element_text(size = 40),
    legend.text = element_text(size = 40),
    legend.title = element_text(size = 40)
)

## Effects of one year of hatchery input 
Here we see how operating a hatchery for one year only impacts the ecosystem. We start from a steady state, then add one year of hatchery input (instantaneously) and see how the system evolves in time.

### Steady state without a hatchery:

In [ ]:
# define params + starting steady state
params <- newSingleSpeciesParams()
species_params(params)

params <- steady(params)

# params <- setColours(params, list("Resource" = "db6e45", "Target Species" = "349bfc"))

t <- 0.0
fig <- plotSpectra(params) #+ ggtitle("Time = 0.00")

# fig <- fig %>% layout(autosize = F)
ggsave("single species/nop-steady-0.00.png", dpi=300, width = 6, height = 4)
ggplotly(fig)

### Adding hatchery input

Notes on hatchery implementation:
- user specifies `hatchery_params` dataframe which contains:
  - `species` - species to add biomass to (can be multiple)
  - `mu` - mean size of added species distribution
  - `sigma` - standard deviation of normally distributed biomass addition
  - `annual_N` - the number of individuals of the species to add to the system annually (across the specified size distribution)
- hatchery adds addtional biomass in a normal distribution over the course of a year
- default version uses constant addition over the course of the whole year (may be altered by changing the definition of `hatchery_dynamics`)

Here, we add the hatchery input for a single year at the start of the simulation. We plot the biomass spectrum for various times during the evolution of the system to see how the spectrum evolves from this initial condition.

In [ ]:
# add hatchery input
hatchery_params = data.frame(species="Target species", mu=4, sigma=0.2, annual_N=0.01)
View(hatchery_params) # display hatchery params

sim_original <- projectHatchery(params)
sim <- projectHatchery(params, t_max=1, t_save=1, dt=1)
plotlySpectra(sim) # plot biomass spectra after hatchery input

Below, we observe that the total number of idndividuals has changed as expected (as specified in `hatchery_params`):

In [ ]:
View(hatchery_params)
getN(sim) # observe change in total number of individuals between states

In [ ]:
# project this simulation forwards in time
sim <- project(sim, t_max=10, t_save=0.05, dt=0.05)

# display biomass spectra at various times through the simulation
times = c(1, 1.2, 2.2, 10) # expected to be explicit time steps of `sim2`
for (t in times) {
    # create plot of spectra at time `t`
    p <- plotSpectra(sim, time_range=t, linetype=c(1,2)) #+ ggtitle(sprintf("Time = %.2f", t)) 
    ggsave(sprintf("single species/nop-steady-%.2f.png",t), dpi=300, width = 6, height = 4)

    View(ggplotly(p))
}

In [ ]:
# # save a series of plots to generate an animation ----

# # get vector of times from projection
# times <- getTimes(sim)
# # remove first time from animation (steady state before hatchery input)
# times <- times[2:length(times)]

# for (t in times) {
#     print(sprintf("plots/spectra/%.2f.png", t)) # debug

#     # create plot of spectra at time `t`
#     p <- plotSpectra(sim, time_range=t) + ggtitle(sprintf("Time = %.2f", t))

#     # save the plots
#     ggsave(sprintf("plots/spectra/%.2f.png", t),
#         plot = p,
#         device="png",
#         dpi=320,
#         scale=2)
# }

# #! to compile gif run the following in bash (with ImageMagick installed) 
# #! `convert -resize 30% -delay 15 -loop 0 *.png spectra.gif`

Animation:
<!-- <details>
<summary>Command</summary>
<code>convert -resize 30% -delay 15 -loop 0 *.png spectra.gif</code>
</details>   -->

<!-- ! include gif here -->
<!-- ![Biomass spectra animation](plots/spectra.gif "spectra.gif")<> -->

Above, we see that the added population grow, resulting in advection across the biomass spectrum. We also see the peak height reduce in height as the distribution diffuses along the right of the spectrum (only to the right since there is no concept of shrinking), the reduction in peak height results partly from mortality (only external in this model) and *partly from the dependence of growth rate on encounters, meaning that the whole population does not grow at the same rate* **(double check this - does a singlet wave diffuse + advect to the right as expected?)**.
<!-- the biomass projection reenforces this idea since the reduction in the peak is clearly not just due to mortality since -->

After a short time period, we also see the lower end of the spectrum begin to increase. This occurs when the added population grow to beyond `w_mat` and begin to reproduce **(does the diffusion rate slow beyond this weight as a result of converting some biomass into reproduction rather than growth?)**.

#### Biomass projection:

In [ ]:
# plotlyBiomass(sim, start_time=1) # plot after the year of hatchery input
# getBiomass(params)
plotlyBiomassRelative(sim, )
ggsave("single species/nop-biomass.png", dpi=300, width = 6, height = 4)

Comparing the biomass in the initial steady state with no hatchery to the initial and steady state biomasses obtained from the simulation, we see that there is a significant increase in the total biomass as a result of the hatchery.

In [ ]:
df = data.frame(time=getTimes(sim), N=getN(sim)[,])
p <- ggplot(data=df, aes(x=time, y=N, group=1)) + geom_line()
ggplotly(p)

## Long term impact of hatchery

Note that in the example above, we increased the steady state biomass. For this simple system, the predator/prey size ratio preference is large enough that there is no (or negligible) predation occuring, hence, the equilibrium state is only limited by the external mortality rate and the resource assimilation rate.

### Current system
We expect this behaviour to continue under the influence of a hatchery, so we will likely continue to see an increase in the total biomass of the system (which will eventually become negligible on the size of the system). Below we test this for the constant yearly addition case. It is here where the temporal dynamics of the hatchery are likely to come into play.

In [ ]:
params <- newSingleSpeciesParams()
params <- steady(params)

sim2 <-  projectHatchery(params, t_max = 100, t_save=0.25, dt=0.1)

fig <- plotBiomass(sim2)

ggsave("single species/nop-continuous-biomass.png", dpi=300, width = 6, height = 4)
ggplotly(fig)

In [ ]:
# display biomass spectra at various times through the simulation
times = c(0, 0.25, 0.5, 1, 5, 100) # expected to be explicit time steps of `sim2`
for (t in times) {
    # create plot of spectra at time `t`
    p <- plotSpectra(sim2, time_range=t)# + ggtitle(sprintf("Time = %.2f", t))
    ggsave(sprintf("single species/nop-cont-%.2f.png",t), dpi=300, width = 6, height = 4)

    View(ggplotly(p))
}

In [ ]:
# plot number of individuals projection
df = data.frame(time=getTimes(sim2), N=getN(sim2)[,])
p <- ggplot(data=df, aes(x=times, y=N, group=1)) + geom_line()
ggplotly(p)

Above, we see that the biomass of the system does monotonically increase with diminishing effect as expected. The biomass spectrum gradually evens out to the same shape as it naturally takes (only with higher total biomass).

### Cannibalistic model
In the above models, our target species is parameterized with no interaction (`interaction` matrix = [0]) between members of the target species such a way that no cannibalism occurs, i.e. all mortality is due to "external" sources (mizer approximation for deaths which aren't caused by predation or fishing).

Predation is a key component to ecosystem models, as such it is worth investigating the impact of a hatchery on a single cannibalistic species to investigate how predation changes the dynamics of a hatchery model. Here, we will (initially) investigate the dynamics of a model using our approximated lobster species (previously parameterized):

In [ ]:
params3 <- newSingleSpeciesParams()
# increase interaction with same species (previously 0)
params3 <- setInteraction(params3, 1)
# decrease interaction with resource (previously 1)
species_params(params3)$interaction_resource <- 0.8

params3 <- steady(params3)

species_params(params)
getN(params3)
fig <- plotSpectra(params3)
ggsave(sprintf("single species/p-0.0.png",t), dpi=300, width = 6, height = 4)

View(ggplotly(fig))

In [ ]:
plotlyPredMort(params3)
ggplotly(plotDiet(params3))

In [ ]:
# project one year of hatchery input (using same hatchery_params as before)
sim3 <- projectHatchery(params3, t_max = 1, t_save = 1, dt = 1)
plotlySpectra(sim3)

In [ ]:
# project this simulation forwards in time
sim3 <- project(sim3, t_max=10, t_save=0.05, dt=0.05)


fig <- plotBiomass(sim3)
ggsave(sprintf("single species/p-biomass.png",t), dpi=300, width = 6, height = 4)
ggplotly(fig)

In [ ]:
# display biomass spectra at various times through the simulation
times = c(1, 1.1, 1.2, 1.5, 2.2, 2.6, 10) # expected to be explicit time steps of `sim2`
for (t in times) {
    # create plot of spectra at time `t`
    p <- plotSpectra(sim3, time_range=t) #+ ggtitle(sprintf("Time = %.2f", t))
    ggsave(sprintf("single species/p-%.2f.png",t), dpi=300, width = 6, height = 4)

    # View(ggplotly(p))
}

#### Continuous hatchery action

In [ ]:
sim4 <-  projectHatchery(params3, t_max = 100, t_save=0.25, dt=0.05)

p <- plotBiomass(sim4)
ggsave(sprintf("single species/p-cont-biomass.png",t), dpi=300, width = 6, height = 4)

View(ggplotly(p))

In [ ]:
# display biomass spectra at various times through the simulation
times = c(0, 0.25, 0.5, 1, 5) # expected to be explicit time steps of `sim2`
for (t in times) {
    # create plot of spectra at time `t`
    p <- plotSpectra(sim4, time_range=t)# + ggtitle(sprintf("Time = %.2f", t))
    ggsave(sprintf("single species/p-cont-%.2f.png",t), dpi=300, width = 6, height = 4)

    View(ggplotly(p))
}

Above, we see that the biomass density drops to practically nothing before the hatchery peak due to predation from the larger individuals - this may be due to adding an unrealistcally large number of individuals or unrealistic predation parameters - either way, it is worth noting that this situation is possible in a model and implies that all individuals which hatch naturally die before growing to maturity due to cannibalism (a less-than ideal situation).

Below we demonstrate what happens if we stop the hatchery's input from the new steadystate, showing that the model tends back to the no-hatchery steady state (indicating resilience to this kind of disruption).

In [ ]:
sim4 <- project(sim4, t_max = 50, t_save=0.5, dt=0.1)

plotlyBiomass(sim4)

In [ ]:
# display biomass spectra at various times through the simulation
times = c(100, 101, 104, 108) # expected to be explicit time steps of `sim2`
for (t in times) {
    # create plot of spectra at time `t`
    p <- plotSpectra(sim4, time_range=t) + ggtitle(sprintf("Time = %.2f", t))

    View(ggplotly(p))
}